In [ ]:
# Copyright 2026 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Spot-Checking Places Insights Data with Functions and Sample Place IDs

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/googlemaps-samples/insights-samples/blob/main/places_insights/notebooks/spot_check_results/places_insights_spot_check_results_using_functions.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fgooglemaps-samples%2Finsights-samples%2Fmain%2Fplaces_insights%2Fnotebooks%2Fspot_check_results%2Fplaces_insights_spot_check_results_using_functions.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/bigquery/import?url=https://github.com/googlemaps-samples/insights-samples/blob/main/places_insights/notebooks/spot_check_results/places_insights_spot_check_results_using_functions.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/bigquery/v1/32px.svg" alt="BigQuery Studio logo"><br> Open in BigQuery Studio
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/googlemaps-samples/insights-samples/blob/main/places_insights/notebooks/spot_check_results/places_insights_spot_check_results_using_functions.ipynb">
      <img width="32px" src="https://upload.wikimedia.org/wikipedia/commons/9/91/Octicons-mark-github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

### Overall Goal

This notebook demonstrates a workflow for spot-checking Places Insights data. It starts with a high-level statistical query to find restaurant density and then **directly visualizes both the high-level density and ground-truth sample locations from the city's busiest areas on a single, combined map.**

### Key Technologies Used

*   **[Places Insights](https://developers.google.com/maps/documentation/placesinsights):** To provide the Places Data and Place Count Function.
*   **[BigQuery](https://cloud.google.com/bigquery):** To run the `PLACES_COUNT_PER_H3` function, which provides aggregated place counts and `sample_place_ids`.
*   **[Google Maps Place Details API](https://developers.google.com/maps/documentation/places/web-service/place-details):** To fetch rich, detailed information (name, address, and a Google Maps link) for the specific `sample_place_ids`.
*   **[Google Maps 2D Tiles](https://developers.google.com/maps/documentation/tile/2d-tiles-overview):** To use Google Maps as the basemap.
*   **Python Libraries:**
    * **[GeoPandas](https://geopandas.org/en/stable/)** for spatial data manipulation.
    * **[Folium](https://python-visualization.github.io/folium/latest/)** for creating the final interactive, layered map.

See [Google Maps Platform Pricing](https://mapsplatform.google.com/intl/en_uk/pricing/) For API costs assocated with running this notebook.

### The Step-by-Step Workflow

1.  **Query Aggregated Data:** We begin by querying BigQuery to count all highly-rated, operational restaurants within a **5km radius of Central London**. We group them into H3 hexagonal cells at **resolution 9**. This query provides the statistical foundation for our analysis and, crucially, a list of `sample_place_ids` for each cell.

2.  **Identify Hotspots & Fetch Details:** The notebook then **automatically** identifies the 20 busiest H3 cells. It consolidates the `sample_place_ids` from all of these top hotspots into a single master list and uses the Places API to fetch detailed information for each one.

3.  **Create a Combined Visualization:** In the final step, we generate a single, layered map.
    *   The **base layer** is a choropleth "heatmap" showing restaurant density across the search area.
    *   The **top layer** displays individual pins for all the sample restaurants from the top 20 hotspots, providing a direct, ground-level view of the locations that make up the aggregated counts. Each pin's popup includes a link to open the location directly in Google Maps.

### **How to Use This Notebook**

1.  **Set Up Secrets:** Before you begin, you must configure two secrets in the Colab "Secrets" tab (the **key icon** on the left menu):
    *   `GCP_PROJECT`: Your Google Cloud Project ID with access to Places Insights.
    *   `GMP_API_KEY`: Your Google Maps Platform API key. Ensure the **Maps Tile API** is enabled for this key in your GCP console.

2.  **Run the Cells:** Once the secrets are set, simply run the cells in order from top to bottom. Each visualization will appear as the output of its corresponding code cell.

In [ ]:
# Install the Google Maps Places Client Library
!pip install --quiet google-maps-places

In [ ]:
# Import libraries
from google.cloud import bigquery
from google.colab import auth, userdata, data_table
from google.api_core import exceptions

from google.maps import places_v1

import requests

import geopandas as gpd
import shapely
from shapely import wkt
import sys

import pandas as pd

# Import the mapping libraries
import folium
import mapclassify # Used by .explore() for data classification
import xyzservices # Provides tile layers

In [ ]:
# @title Configure GCP Authentication
# @markdown This part securely gets your GCP Project ID and initializes a BigQuery Client.
GCP_PROJECT_SECRET_KEY_NAME = "GCP_PROJECT" #@param {type:"string"}
GCP_PROJECT_ID = None

if "google.colab" in sys.modules:
    try:
        GCP_PROJECT_ID = userdata.get(GCP_PROJECT_SECRET_KEY_NAME)
        if GCP_PROJECT_ID:
            print(f"Authenticating to GCP project: {GCP_PROJECT_ID}")
            auth.authenticate_user(project_id=GCP_PROJECT_ID)
        else:
            raise ValueError(f"Could not retrieve GCP Project ID from secret named '{GCP_PROJECT_SECRET_KEY_NAME}'. "
                             "Please make sure the secret is set in your Colab environment.")
    except userdata.SecretNotFoundError:
        raise ValueError(f"Secret named '{GCP_PROJECT_SECRET_KEY_NAME}' not found. "
                         "Please create it in the 'Secrets' tab (key icon) in Colab.")

client = bigquery.Client(project=GCP_PROJECT_ID)

In [ ]:
# @title Get GMP API Key

API_KEY_SECRET_NAME = "GMP_API_KEY" #@param {type:"string"}

# Initialize a variable to hold our key.
gmp_api_key = None

try:
  # Attempt to retrieve the secret value using its name.
  gmp_api_key = userdata.get(API_KEY_SECRET_NAME)
  print("Successfully retrieved API key.")

except userdata.SecretNotFoundError:
  raise ValueError(f"Secret named '{API_KEY_SECRET_NAME}' not found. "
                         "Please create it in the 'Secrets' tab (key icon) in Colab.")

In [ ]:
# @title Maps Backend Initialization: Session, Copyright & Assets
# @markdown This cell manages the Maps Tiles API handshake. It performs the following steps:
# @markdown 1.  **Session Creation:** Authenticates and requests a "Roadmap" session for the target region.
# @markdown 2.  **Attribution Fetching:** Queries the API for the specific copyright text required for the configured viewport.
# @markdown 3.  **Asset Preparation:** Generates the compliant HTML for the Google Maps logo overlay.

# --- Configuration ---
# We use the center point from the SQL query (Charing Cross) to make sure
# the attribution we fetch matches the area we are mapping.
CENTER_LAT = 51.5074
CENTER_LNG = -0.1276
ZOOM_LEVEL = 12

# --- 1. Create Google Maps Session ---
print("🗺️ Initializing Google Maps Session...")
session_url = f"https://tile.googleapis.com/v1/createSession?key={gmp_api_key}"
headers = {"Content-Type": "application/json"}
payload = {
    "mapType": "roadmap",
    "language": "en-GB",
    "region": "GB"
}

try:
    response = requests.post(session_url, json=payload, headers=headers)
    response.raise_for_status()
    session_token = response.json().get("session")
    print(f"✅ Session Token acquired: {session_token[:10]}...")
except Exception as e:
    raise RuntimeError(f"Failed to initialize Google Maps session: {e}")

# --- 2. Fetch Dynamic Attribution ---
# We calculate a small dynamic bounding box around our center point
# to fetch the correct copyright for London.
delta = 0.05 # Roughly 5km buffer
north, south = CENTER_LAT + delta, CENTER_LAT - delta
east, west = CENTER_LNG + delta, CENTER_LNG - delta

viewport_url = (
    f"https://tile.googleapis.com/tile/v1/viewport?key={gmp_api_key}"
    f"&session={session_token}"
    f"&zoom={ZOOM_LEVEL}"
    f"&north={north}&south={south}"
    f"&west={west}&east={east}"
)

try:
    vp_response = requests.get(viewport_url)
    vp_response.raise_for_status()
    google_attribution = vp_response.json().get('copyright', 'Map data © Google')
    print("✅ Attribution fetched.")
except Exception as e:
    print(f"⚠️ Warning: Could not fetch attribution ({e}). Defaulting.")
    google_attribution = "Map data © Google"

# --- 3. Construct Tile URL ---
# This variable will be passed to 'tiles=' in folium
google_tiles_url = f"https://tile.googleapis.com/v1/2dtiles/{{z}}/{{x}}/{{y}}?session={session_token}&key={gmp_api_key}"

# --- 4. Construct Logo HTML ---
# This HTML block will be injected into the map root
logo_url = "https://maps.gstatic.com/mapfiles/api-3/images/google_white3.png"
google_logo_html = f"""
    <div style="
        position: fixed;
        bottom: 35px;
        left: 10px;
        z-index: 9999;
        font-size: 0;
        pointer-events: none;
    ">
        <img src="{logo_url}" alt="Google Maps" style="height: 18px;">
    </div>
"""
print("✅ Logo HTML prepared.")

In [ ]:
# @title Query Aggregated Restaurant Data
# @markdown This query selects operational restaurants with a rating > 3.5 within a 5km radius of Central London.
# @markdown The results are aggregated into H3 cells (Resolution 9) and saved to the dataframe `df_restaurants_in_london`.
%%bigquery df_restaurants_in_london --project $GCP_PROJECT_ID

DECLARE geo GEOGRAPHY;

-- Create a polygon: A 5km radius around Central London (Charing Cross)
SET geo = ST_BUFFER(ST_GEOGPOINT(-0.1276, 51.5074), 5000);

SELECT *
FROM `places_insights___gb.PLACES_COUNT_PER_H3`(
    JSON_OBJECT(
        'geography', geo,
        'h3_resolution', 9,
        'types', ['restaurant'],
        'business_status', ['OPERATIONAL'],
        'min_rating', 3.5
    )
);

In [ ]:
# 1. Convert the WKT string column into geometry objects
# We use apply(wkt.loads) to parse the string representation of the polygons
df_restaurants_in_london['geography'] = df_restaurants_in_london['geography'].apply(wkt.loads) # type: ignore

# 2. Convert the standard DataFrame to a GeoDataFrame
gdf_restaurants_in_london = gpd.GeoDataFrame(
    df_restaurants_in_london, # type: ignore
    geometry='geography',
    crs='EPSG:4326'
)

print(f"Successfully processed {len(gdf_restaurants_in_london)} cells.")
print("Displaying top 5 rows:")
display(gdf_restaurants_in_london.head())

### Visualizing Restaurant Density on a Heatmap

The code below uses the GeoDataFrame to generate a heatmap. Here's what it shows:

*   **Choropleth Map:** Each H3 hexagon on the map is colored based on the number of operational restaurants it contains.
*   **Color Scale:** The map uses a yellow-to-red color scale (`YlOrRd`). Lighter, yellow areas have fewer restaurants, while darker, red areas represent the densest hotspots.
*   **Interactivity:** You can hover over any hexagon to see its unique H3 index and the exact restaurant count. Clicking on a hexagon will open a popup with all the data for that cell.

This visualization gives us an immediate and intuitive understanding of where the major dining hubs are located throughout the city.

**Note:** This cell uses [2D Map Tiles](https://developers.google.com/maps/documentation/tile/2d-tiles-overview). Please review the documentation for pricing.

In [ ]:
# Check if dataframe exists
if 'gdf_restaurants_in_london' in locals() and not gdf_restaurants_in_london.empty:

    # Define tooltips
    restaurant_tooltip_cols = ['h3_cell_index', 'count']

    # 1. Create the map using .explore()
    # Note how clean this is now: we just pass the pre-calculated URL and Attribution
    london_restaurants_map = gdf_restaurants_in_london.explore(
        column="count",
        cmap="YlOrRd",
        scheme="NaturalBreaks",
        tooltip=restaurant_tooltip_cols,
        popup=True,
        tiles=google_tiles_url,       # <--- Injected from Setup Cell
        attr=google_attribution,      # <--- Injected from Setup Cell
        style_kwds={"stroke": True, "color": "black", "weight": 0.2, "fillOpacity": 0.7}
    )

    # 2. Inject the Google Logo
    # This adds the HTML div on top of the map container
    london_restaurants_map.get_root().html.add_child(folium.Element(google_logo_html))

    # 3. Display
    display(london_restaurants_map)
else:
    print("DataFrame 'gdf_restaurants_in_london' not found. Please run the BigQuery cell first.")

### Identify Top Hotspots and Consolidate Place IDs

Now that we have the density data for all of London, we will focus our analysis on the 20 busiest areas.

The code below isolates these top 20 H3 cells and extracts up to 10 `sample_place_ids` from each one. It then displays a summary table of these hotspots before consolidating all the IDs into a single master list for analysis. This list will be used in the next step to fetch detailed information for each location.

In [ ]:
# Isolate the top 20 H3 cells with the highest restaurant counts.
print(f"Identifying the top 20 busiest H3 cells from the {len(gdf_restaurants_in_london)} total cells...")
top_20_cells_df = gdf_restaurants_in_london.sort_values(by='count', ascending=False).head(20).reset_index(drop=True)
print("Top 20 cells identified.")

# For each of the top 20 cells, take the first 10 sample_place_ids.
# The .apply() method performs this slicing operation on each row's list individually.
print("Extracting up to 10 sample Place IDs from each of the top 20 cells...")
sliced_ids_series = top_20_cells_df['sample_place_ids'].apply(lambda id_list: id_list[:10])

# --- Create and display the summary table ---
print("Generating summary table of top hotspots...")
summary_df = pd.DataFrame({
    'H3 Cell Index': top_20_cells_df['h3_cell_index'],
    'Total Places in Cell': top_20_cells_df['count'],
    'Sample IDs to Analyze': sliced_ids_series.apply(len)
})
display(summary_df)


# Consolidate all the sliced lists into a single series.
# The .explode() function creates a new row for each Place ID.
all_place_ids_series = sliced_ids_series.explode()

# Get a final list of unique Place IDs to process.
place_ids_to_process = all_place_ids_series.unique().tolist()

print(f"\nConsolidated a total of {len(place_ids_to_process)} unique sample Place IDs to analyze.")

# Display the first 5 IDs as a sample
print("\nSample of Place IDs to be processed:")
print(place_ids_to_process[:5])

### Fetch Details for Consolidated Place IDs

With our consolidated list of unique Place IDs, we now use the Google Maps Places API to fetch rich, detailed information for each location.

The script below will loop through each ID and retrieve its name, address, user rating, and latitude/longitude coordinates. All of this information is then compiled into a single DataFrame, `details_df`, which will power our final, combined map visualization.

**Note:** This cell uses [Place Details API](https://developers.google.com/maps/documentation/places/web-service/place-details). Please review the documentation for pricing.

In [ ]:
# Check if the list of Place IDs from Phase 1 exists.
if 'place_ids_to_process' in locals() and place_ids_to_process:

    places_client = places_v1.PlacesClient()
    if places_client:
        # Loop through the list of Place IDs and fetch details.
        place_details_list = []
        fields_to_request = "formattedAddress,location,displayName,googleMapsUri"

        total_ids = len(place_ids_to_process)
        print(f"\nFetching details for {total_ids} unique Place IDs...")

        for i, place_id in enumerate(place_ids_to_process):
            if i > 0 and i % 50 == 0:
                print(f"  ...processed {i} of {total_ids} IDs.")

            try:
                request = {"name": f"places/{place_id}"}
                response = places_client.get_place(
                    request=request,
                    metadata=[("x-goog-fieldmask", fields_to_request)]
                )

                place_details_list.append({
                    "Name": response.display_name.text,
                    "Address": response.formatted_address,
                    "Place ID": place_id,
                    "Latitude": response.location.latitude,
                    "Longitude": response.location.longitude,
                    "Google Maps URI": response.google_maps_uri
                })
            except exceptions.GoogleAPICallError as e:
                print(f"  - Warning: Could not fetch details for Place ID '{place_id}': {e.message}")

        # Convert the list of details into a pandas DataFrame.
        if place_details_list:
            print(f"\nSuccessfully fetched details for {len(place_details_list)} places.")
            details_df = pd.DataFrame(place_details_list)

            # Define which columns we want to show in the summary table.
            columns_to_display = ["Name", "Address", "Place ID"]

            print("Here is a sample of the retrieved data (Google Maps URI is hidden):")
            # Display only the selected columns from the head of the DataFrame.
            display(details_df[columns_to_display].head())
        else:
            print("\nCould not fetch details for any of the sample Place IDs.")
            details_df = pd.DataFrame()

else:
    print("The 'place_ids_to_process' list does not exist or is empty. "
          "Please run the previous cell to generate the list of IDs first.")

### Create the Combined Map

This is the final step where we bring all our analysis together into a single visualization.

The code below first creates the restaurant density heatmap, coloring each H3 cell based on the number of qualifying restaurants. Then, it iterates through the restaurant data we fetched in the previous step and overlays a  pin for each restaurant onto the map.

The result is a layered map that shows both the high-level "hotspots" and the ground-truth, individual places that make up those dense areas.

**Note:** This cell uses [2D Map Tiles](https://developers.google.com/maps/documentation/tile/2d-tiles-overview). Please review the documentation for pricing.

In [ ]:
# Check if required DataFrames exist
if 'gdf_restaurants_in_london' in locals() and not gdf_restaurants_in_london.empty and 'details_df' in locals() and not details_df.empty:

    restaurant_tooltip_cols = ['h3_cell_index', 'count']

    # 1. Generate base choropleth map
    print("Generating base choropleth map...")
    combined_map = gdf_restaurants_in_london.explore(
        column="count",
        cmap="YlOrRd",
        scheme="NaturalBreaks",
        tooltip=restaurant_tooltip_cols,
        popup=True,
        tiles=google_tiles_url,       # <--- Injected from Setup Cell
        attr=google_attribution,      # <--- Injected from Setup Cell
        style_kwds={"stroke": True, "color": "black", "weight": 0.2, "fillOpacity": 0.7}
    )

    # 2. Iterate and add markers (Logic unchanged)
    print(f"Adding {len(details_df)} individual restaurant markers...")
    for index, row in details_df.iterrows():
        lat, lon = row['Latitude'], row['Longitude']

        if pd.isna(lat) or pd.isna(lon): continue

        name = str(row['Name']).replace('`', "'") if pd.notna(row['Name']) else "Unnamed Place"
        address = str(row['Address']) if pd.notna(row['Address']) else ""
        uri = str(row['Google Maps URI']) if pd.notna(row['Google Maps URI']) else "#"

        popup_html = f"""
        <b>{name}</b><br>
        <hr style="margin: 4px 0;">
        {address}<br><br>
        <a href="{uri}" target="_blank">View on Google Maps</a>
        """

        folium.Marker(
            location=[lat, lon],
            tooltip=name,
            popup=folium.Popup(popup_html, max_width=300),
            icon=folium.Icon(color='blue', icon='utensils', prefix='fa')
        ).add_to(combined_map)

    # 3. Inject the Google Logo
    combined_map.get_root().html.add_child(folium.Element(google_logo_html))

    # 4. Display
    print("Displaying combined map.")
    display(combined_map)

else:
    print("Required DataFrames not found. Ensure previous cells ran successfully.")